이 자료는 위키독스 '딥 러닝을 이용한 자연어 처리 입문'의 FastText 튜토리얼 자료입니다. 해당 코드에 대한 설명은 오프라인 출판물이나 강의에서 확인할 수 있습니다.

In [1]:
# Colab에 Mecab 설치 (3분 정도 소요)
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 5.67 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 40.7 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-10-18 06:21:26--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response.

In [20]:
!pip install mecab
# 설치 후 런타임 다시 시작

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mecab: filename=mecab-0.996.3-cp310-cp310-linux_x86_64.whl size=167492 sha256=f70c7836bfe1faae44955ba5bd534517c3e16f927cf7e4c516b495993ca36daf
  Stored in directory: /root/.cache/pip/wheels/10/1a/b2/8c0e17d02a12c7b46e3065f55aa7555d173a5b020110717829
Successfully built mecab


hgtk : https://github.com/bluedisk/hangul-toolkit

In [2]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

In [3]:
# fasttext 설치 (1분 정도 소요)
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3962, done.
remote: Counting objects: 100% (1021/1021), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 3962 (delta 913), reused 874 (delta 857), pack-reused 2941
Receiving objects: 100% (3962/3962), 8.26 MiB | 15.78 MiB/s, done.
Resolving deltas: 100% (2520/2520), done.
/content/Mecab-ko-for-Google-Colab/fastText
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pt

# 1. 데이터 로드

In [9]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
from konlpy.tag import Mecab

In [10]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7a93786a2d10>)

In [11]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


In [12]:
total_data[:5]

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


# 2. hgtk 튜토리얼

In [13]:
# 한글인지 체크
hgtk.checker.is_hangul('ㄱ')

True

In [9]:
# 한글인지 체크
hgtk.checker.is_hangul('28')

False

In [10]:
# 음절을 초성, 중성, 종성으로 분해
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [11]:
# 초성, 중성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ')

'나'

In [12]:
# 초성, 중성, 종성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')

'남'

In [13]:
# 한글이 아닌 입력에 대해서는 에러 발생.
hgtk.letter.decompose('1')

NotHangulException: ignored

In [14]:
# 결합할 수 없는 상황에서는 에러 발생
hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ')

NotHangulException: ignored

# 3. 데이터 전처리

In [14]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [15]:
word_to_jamo('남동생')

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [16]:
word_to_jamo('여동생')

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [17]:
from konlpy.tag import Mecab
mecab = Mecab()

In [18]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [19]:
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [20]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [21]:
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_list()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

100%|██████████| 200000/200000 [01:08<00:00, 2921.19it/s]


In [22]:
len(tokenized_data)

200000

In [23]:
tokenized_data[0]

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']

In [24]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

In [25]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

# 4. FastText

In [26]:
import fasttext

In [27]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 294908.83 line/s]


In [28]:
# 실행시간 3분 정도 소요
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [29]:
model.save_model("fasttext.bin")

In [30]:
model = fasttext.load_model("fasttext.bin")

In [31]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

array([ 7.38955140e-02, -6.52488172e-01, -9.29627895e-01,  3.87560010e-01,
       -5.11987448e-01, -1.63068861e-01, -1.19321115e-01, -1.33572504e-01,
        2.25782394e-01,  1.48044840e-01,  3.43759775e-01, -5.54470897e-01,
        1.67137310e-01, -6.71108902e-01, -2.25550160e-01, -2.01829627e-01,
        1.03950095e+00,  3.44124138e-02, -9.37988162e-01, -3.69810641e-01,
        3.34442966e-02, -2.22831786e-01, -9.03086722e-01, -6.76649034e-01,
        6.09328926e-01, -8.61241072e-02, -4.12651718e-01,  9.01440561e-01,
        1.16067541e+00, -4.20458019e-01,  7.15754688e-01, -3.68723989e-01,
        6.35134876e-01, -1.03655756e+00, -5.99228330e-02,  4.34701473e-01,
       -1.05250275e+00, -2.24264607e-01, -5.37342131e-01, -4.29082870e-01,
       -1.89368010e-01,  5.41120887e-01, -1.62168309e-01,  9.38618422e-01,
       -2.61361808e-01, -8.64060402e-01, -2.95687288e-01,  6.60960898e-02,
       -2.15251520e-01, -3.96316499e-02,  2.79447827e-02,  1.20972610e+00,
       -4.22157884e-01, -

In [32]:
# '남동생'과 가장 비슷한 10개의 단어 출력
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8867998123168945, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8419036269187927, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.768511950969696, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7601867914199829, 'ㅊㅣㄴㄱㅜ-'),
 (0.7493603229522705, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7321819067001343, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.7091388702392578, 'ㅇㅓㄴㄴㅣ-'),
 (0.7076253294944763, 'ㅈㅗ-ㅋㅏ-'),
 (0.7050490379333496, 'ㄴㅏㅁㅇㅏ-'),
 (0.7041964530944824, 'ㅎㅏㄱㅅㅐㅇ')]

In [33]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [34]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))

[('동생', 0.8867998123168945), ('남친', 0.8419036269187927), ('생일', 0.768511950969696), ('친구', 0.7601867914199829), ('남편', 0.7493603229522705), ('중학생', 0.7321819067001343), ('언니', 0.7091388702392578), ('조카', 0.7076253294944763), ('남아', 0.7050490379333496), ('학생', 0.7041964530944824)]


In [35]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동쉥'), k=10)))

[('남동생', 0.8938823342323303), ('남친', 0.8063156008720398), ('남매', 0.7580035924911499), ('남녀', 0.7160884141921997), ('남겼', 0.7126834392547607), ('남짓', 0.7101874351501465), ('남김', 0.7022176384925842), ('남아', 0.7017832398414612), ('남긴', 0.6921391487121582), ('나눴', 0.6887045502662659)]


In [36]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동셍ㅋ'), k=10)))

[('남동생', 0.808269202709198), ('남친', 0.7169588208198547), ('남녀', 0.6457264423370361), ('남매', 0.6320556998252869), ('남김', 0.6282046437263489), ('남짓', 0.6192587614059448), ('남아', 0.6185793280601501), ('남여', 0.6110441088676453), ('남길', 0.6077744364738464), ('남겼', 0.5993062257766724)]


In [37]:
print(transform(model.get_nearest_neighbors(word_to_jamo('난동생'), k=10)))

[('남동생', 0.8563234806060791), ('난생', 0.8343001008033752), ('동생', 0.7836980819702148), ('남편', 0.7570983171463013), ('중학생', 0.7456666231155396), ('남아', 0.7411960363388062), ('남친', 0.7363025546073914), ('고등학생', 0.734373152256012), ('학생', 0.7134009003639221), ('나눴', 0.6956930160522461)]


In [38]:
print(transform(model.get_nearest_neighbors(word_to_jamo('낫동생'), k=10)))

[('남동생', 0.923650860786438), ('동생', 0.8989774584770203), ('중학생', 0.7657902836799622), ('남편', 0.7462917566299438), ('남친', 0.7409523725509644), ('친구', 0.7339593172073364), ('학생', 0.7311184406280518), ('평생', 0.7121482491493225), ('난생', 0.7112301588058472), ('고등학생', 0.7097588777542114)]


In [39]:
print(transform(model.get_nearest_neighbors(word_to_jamo('납동생'), k=10)))

[('남동생', 0.9045630097389221), ('동생', 0.8629793524742126), ('남편', 0.7735514044761658), ('고등학생', 0.7678461670875549), ('중학생', 0.7638423442840576), ('남친', 0.7637498378753662), ('난생', 0.7497016787528992), ('초등학생', 0.733350932598114), ('학생', 0.7289005517959595), ('남아', 0.717134416103363)]


In [40]:
print(transform(model.get_nearest_neighbors(word_to_jamo('냚동생'), k=10)))

[('동생', 0.9670435786247253), ('남동생', 0.9067971110343933), ('친구', 0.7985002994537354), ('조카', 0.782721757888794), ('중학생', 0.7711272835731506), ('언니', 0.7668958306312561), ('학생', 0.7641061544418335), ('생일', 0.7586812973022461), ('딸', 0.7351921200752258), ('신랑', 0.7289538979530334)]


In [41]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품질'), k=10)))

[('품질', 0.8527448773384094), ('음질', 0.8273373246192932), ('땜질', 0.749847412109375), ('찜질', 0.7017661929130554), ('사포질', 0.690645694732666), ('퀄리티', 0.6823137998580933), ('성질', 0.6739238500595093), ('군것질', 0.6710801124572754), ('고감', 0.6515473127365112), ('다림질', 0.6484922170639038)]


In [42]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품쥘'), k=10)))

[('고품질', 0.827895998954773), ('재고품', 0.7614164352416992), ('소모품', 0.7155380249023438), ('재품', 0.7113603353500366), ('고퀄', 0.7096031904220581), ('반제품', 0.7033214569091797), ('중고품', 0.7022567391395569), ('소지품', 0.6736356616020203), ('소품', 0.6681662201881409), ('제품', 0.665552020072937)]


In [43]:
print(transform(model.get_nearest_neighbors(word_to_jamo('노품질'), k=10)))

[('고품질', 0.8990438580513), ('품질', 0.8635608553886414), ('음질', 0.8042033910751343), ('땜질', 0.7227952480316162), ('퀄리티', 0.7056160569190979), ('찜질', 0.6772367358207703), ('성질', 0.656897246837616), ('사포질', 0.6501311659812927), ('화질', 0.6423678994178772), ('소모품', 0.6337369680404663)]


In [44]:
print(transform(model.get_nearest_neighbors(word_to_jamo('보품질'), k=10)))

[('고품질', 0.8281571269035339), ('품질', 0.7812762260437012), ('음질', 0.755413293838501), ('보풀', 0.7372139692306519), ('재질', 0.6818247437477112), ('땜질', 0.6794549226760864), ('바느질', 0.6642733812332153), ('사포질', 0.6634398698806763), ('퀄리티', 0.6586447954177856), ('찜질', 0.6514385342597961)]


In [45]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품'), k=10)))

[('반제품', 0.8946834802627563), ('완제품', 0.8753888607025146), ('상품', 0.8558050990104675), ('타제품', 0.8530835509300232), ('재품', 0.833004355430603), ('제풍', 0.8068342208862305), ('최상품', 0.7935329079627991), ('화학제품', 0.7865867614746094), ('중품', 0.7659172415733337), ('재고품', 0.7596505880355835)]


In [46]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품ㅋ'), k=10)))

[('제품', 0.8544304966926575), ('완제품', 0.7478705048561096), ('반제품', 0.7303435802459717), ('제풍', 0.7238841652870178), ('상품', 0.7225798964500427), ('타제품', 0.7185640335083008), ('최상품', 0.7120945453643799), ('재품', 0.6622195839881897), ('화학제품', 0.6592483520507812), ('성품', 0.6567641496658325)]


In [47]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품^^'), k=10)))

[('제품', 0.9405442476272583), ('제풍', 0.8298900723457336), ('반제품', 0.7996130585670471), ('타제품', 0.7876418828964233), ('상품', 0.7842018008232117), ('완제품', 0.783805251121521), ('최상품', 0.7492274045944214), ('재품', 0.7145487666130066), ('화학제품', 0.6996554136276245), ('상품명', 0.6664127111434937)]


In [48]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제푼ㅋ'), k=10)))

[('제풍', 0.6593174934387207), ('제품', 0.6441411972045898), ('완제품', 0.5402679443359375), ('반제품', 0.5387192964553833), ('최상품', 0.5360017418861389), ('타제품', 0.529526948928833), ('젝', 0.5171450972557068), ('상품', 0.5091889500617981), ('젱', 0.4995463192462921), ('제왕', 0.4910176694393158)]
